# 🔭 Historical Entropy Audit

This notebook analyzes historical astronomical events using the Bruno entropy collapse model, estimates fluence at Earth, predicts collapse likelihood, and simulates neutrino detection using `snewpy`.

In [9]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [10]:

historical_events = pd.DataFrame([
    {"name": "SN1987A", "type": "SN", "distance_Mpc": 0.05, "energy_erg": 1e53, "observed_neutrinos": True, "neutrino_detectors": ["Kamiokande-II", "IMB", "Baksan"]},
    {"name": "GW170817", "type": "NS merger", "distance_Mpc": 40, "energy_erg": 1e52, "observed_neutrinos": False, "neutrino_detectors": ["IceCube", "ANTARES", "Super-K"]},
    {"name": "GW150914", "type": "BH merger", "distance_Mpc": 410, "energy_erg": 5e54, "observed_neutrinos": False, "neutrino_detectors": ["IceCube", "ANTARES"]},
    {"name": "SN2016aps", "type": "SN", "distance_Mpc": 350, "energy_erg": 4e53, "observed_neutrinos": False, "neutrino_detectors": ["IceCube", "Super-K"]},
    {"name": "GW190521", "type": "BH merger", "distance_Mpc": 530, "energy_erg": 8e54, "observed_neutrinos": False, "neutrino_detectors": ["IceCube"]}
])


In [11]:

def compute_fluence(E_erg, D_Mpc):
    E_joule = E_erg * 1e-7
    D_meter = D_Mpc * 3.086e22
    return E_joule / (4 * np.pi * D_meter**2)

def required_energy_for_projection(D_Mpc, threshold=1e-5):
    D_meter = D_Mpc * 3.086e22
    E_joule = 4 * np.pi * D_meter**2 * threshold
    return E_joule * 1e7

threshold = 1e-5

def classify_event(row):
    fluence = compute_fluence(row["energy_erg"], row["distance_Mpc"])
    required = required_energy_for_projection(row["distance_Mpc"], threshold)
    return pd.Series({
        "fluence_J_per_m2": fluence,
        "log10_fluence": np.log10(fluence),
        "required_energy_erg": required,
        "log10_required_energy": np.log10(required),
        "collapse_predicted": fluence >= threshold,
        "matches_observation": (fluence >= threshold) == row["observed_neutrinos"]
    })

audit = historical_events.join(historical_events.apply(classify_event, axis=1))


In [12]:

display_cols = ["name", "type", "distance_Mpc", "energy_erg", "collapse_predicted", "observed_neutrinos", "fluence_J_per_m2", "SuperK_IBD_counts"]
audit[display_cols[:-1]]


,name,type,distance_Mpc,energy_erg,collapse_predicted,observed_neutrinos,fluence_J_per_m2
0,SN1987A,SN,0.05,1.000000e+53,True,True,334.239893
1,GW170817,NS merger,40.00,1.000000e+52,True,False,0.000052
2,GW150914,BH merger,410.00,5.000000e+54,True,False,0.000249
3,SN2016aps,SN,350.00,4.000000e+53,True,False,0.000027
4,GW190521,BH merger,530.00,8.000000e+54,True,False,0.000238


In [15]:
column_names = [
    "tau_cc_hr", "E_nu_MeV", "Lnue", "Lnuebar", "Lnux", "Lnuxbar",
    "Lnue_pair", "Lnuebar_pair", "Lnue_beta", "Lnuebar_beta"
]

df_15 = pd.read_csv("totalLuminosity_15SolarMass.dat", comment="#", delim_whitespace=True, names=column_names)
df_30 = pd.read_csv("totalLuminosity_30SolarMass.dat", comment="#", delim_whitespace=True, names=column_names)


C:\Users\ichaj\AppData\Local\Temp\ipykernel_25932\2147816626.py:6: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_15 = pd.read_csv("totalLuminosity_15SolarMass.dat", comment="#", delim_whitespace=True, names=column_names)
C:\Users\ichaj\AppData\Local\Temp\ipykernel_25932\2147816626.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_30 = pd.read_csv("totalLuminosity_30SolarMass.dat", comment="#", delim_whitespace=True, names=column_names)


In [13]:
import matplotlib.pyplot as plt

# Function to plot luminosity vs energy for a given time window
def plot_luminosity_vs_energy(df, title):
    latest_time = df['tau_cc_hr'].min()  # closest to collapse
    earliest_time = df['tau_cc_hr'].max()  # earliest

    fig, ax = plt.subplots(1, 2, figsize=(14, 5), sharey=True)

    # Near-collapse snapshot
    near_collapse = df[df['tau_cc_hr'] < 0.001]
    ax[0].plot(near_collapse["E_nu_MeV"], near_collapse["Lnue"], label="νe")
    ax[0].plot(near_collapse["E_nu_MeV"], near_collapse["Lnuebar"], label="ν̄e")
    ax[0].plot(near_collapse["E_nu_MeV"], near_collapse["Lnux"], label="νx")
    ax[0].set_title("Near Collapse (< 3.6 sec)")
    ax[0].set_xlabel("Neutrino Energy (MeV)")
    ax[0].set_ylabel("Luminosity (1/MeV/s)")
    ax[0].legend()
    ax[0].grid(True)

    # Far-from-collapse snapshot
    far = df[df['tau_cc_hr'] > earliest_time - 0.1]
    ax[1].plot(far["E_nu_MeV"], far["Lnue"], label="νe")
    ax[1].plot(far["E_nu_MeV"], far["Lnuebar"], label="ν̄e")
    ax[1].plot(far["E_nu_MeV"], far["Lnux"], label="νx")
    ax[1].set_title("Far from Collapse (oldest record)")
    ax[1].set_xlabel("Neutrino Energy (MeV)")
    ax[1].legend()
    ax[1].grid(True)

    fig.suptitle(title, fontsize=14)
    plt.tight_layout()
    plt.show()

# Plot for both stars
plot_luminosity_vs_energy(df_15, "Luminosity vs. Energy — 15 Solar Mass Star")
plot_luminosity_vs_energy(df_30, "Luminosity vs. Energy — 30 Solar Mass Star")


NameError: name 'df_15' is not defined

In [16]:
# Combined full Jupyter-ready block: Load both files and plot luminosity vs energy snapshots

import pandas as pd
import matplotlib.pyplot as plt

# Define column headers based on file structure
column_names = [
    "tau_cc_hr", "E_nu_MeV", "Lnue", "Lnuebar", "Lnux", "Lnuxbar",
    "Lnue_pair", "Lnuebar_pair", "Lnue_beta", "Lnuebar_beta"
]

# Load the two datasets
df_15 = pd.read_csv("/mnt/data/totalLuminosity_15SolarMass.dat", comment="#", delim_whitespace=True, names=column_names)
df_30 = pd.read_csv("/mnt/data/totalLuminosity_30SolarMass.dat", comment="#", delim_whitespace=True, names=column_names)

# Plotting function for near vs far collapse snapshots
def plot_luminosity_vs_energy(df, title):
    latest_time = df['tau_cc_hr'].min()  # closest to collapse
    earliest_time = df['tau_cc_hr'].max()  # earliest

    fig, ax = plt.subplots(1, 2, figsize=(14, 5), sharey=True)

    # Near-collapse snapshot (< 0.001 hr = 3.6 sec)
    near_collapse = df[df['tau_cc_hr'] < 0.001]
    ax[0].plot(near_collapse["E_nu_MeV"], near_collapse["Lnue"], label="νe")
    ax[0].plot(near_collapse["E_nu_MeV"], near_collapse["Lnuebar"], label="ν̄e")
    ax[0].plot(near_collapse["E_nu_MeV"], near_collapse["Lnux"], label="νx")
    ax[0].set_title("Near Collapse (< 3.6 sec)")
    ax[0].set_xlabel("Neutrino Energy (MeV)")
    ax[0].set_ylabel("Luminosity (1/MeV/s)")
    ax[0].legend()
    ax[0].grid(True)

    # Far-from-collapse snapshot (beginning of timeline)
    far = df[df['tau_cc_hr'] > earliest_time - 0.1]
    ax[1].plot(far["E_nu_MeV"], far["Lnue"], label="νe")
    ax[1].plot(far["E_nu_MeV"], far["Lnuebar"], label="ν̄e")
    ax[1].plot(far["E_nu_MeV"], far["Lnux"], label="νx")
    ax[1].set_title("Far from Collapse (start of data)")
    ax[1].set_xlabel("Neutrino Energy (MeV)")
    ax[1].legend()
    ax[1].grid(True)

    fig.suptitle(title, fontsize=14)
    plt.tight_layout()
    plt.show()

# Run for both masses
plot_luminosity_vs_energy(df_15, "Luminosity vs. Energy — 15 Solar Mass Star")
plot_luminosity_vs_energy(df_30, "Luminosity vs. Energy — 30 Solar Mass Star")


C:\Users\ichaj\AppData\Local\Temp\ipykernel_25932\3281066733.py:13: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_15 = pd.read_csv("/mnt/data/totalLuminosity_15SolarMass.dat", comment="#", delim_whitespace=True, names=column_names)


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/totalLuminosity_15SolarMass.dat'